# 1. Import packages


In [ ]:
# !pip -q install demoji
!pip -q install torch
!pip -q install torch==2.0.1 torchtext

In [7]:
import re
import demoji
import random
import inflect
import torch
import torchtext


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F

from bs4 import BeautifulSoup
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import TensorDataset, DataLoader, random_split


ModuleNotFoundError: No module named 'torch'

In [4]:
import torch.nn as nn

In [6]:
file_path = '/content/human_chat.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

In [7]:
lines[1:3]

['Human 2: What is your favorite holiday?\n',
 'Human 1: one where I get to meet lots of different people.\n']

In [8]:
len(lines)

1495

In [9]:
def preprocess_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()

    p = inflect.engine()
    demoji.download_codes()

    text = demoji.replace(text, "")

    text = re.sub(r'\b(?:Human 1|Human 2)\b:?', " ", text)

    text = re.sub(r'\b\d+\b', lambda x: p.number_to_words(x.group()), text)

    text = re.sub('[^a-zA-Z\s]', ' ', text)

    text = text.replace(u'\xa0', u' ').replace('\u200a', ' ').strip()

    return text

In [10]:
preprocessed_lines = [preprocess_text(line) for line in lines]

<ipython-input-9-85a20fb71f40>:6: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [11]:
preprocessed_lines[0:5]

['Hi',
 'What is your favorite holiday',
 'one where I get to meet lots of different people',
 'What was the most number of people you have ever met during a holiday',
 'Hard to keep a count  Maybe twenty five']

In [12]:
tokenizer = get_tokenizer('basic_english')
tokenizer_conv = [tokenizer(conv) for conv in preprocessed_lines]

In [13]:
tokenizer_conv[3]

['what',
 'was',
 'the',
 'most',
 'number',
 'of',
 'people',
 'you',
 'have',
 'ever',
 'met',
 'during',
 'a',
 'holiday']

In [14]:
feature_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenizer_conv,
    min_freq = 1,
    specials = ['<pad>', '<oov>'],
    special_first = True
)

target_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenizer_conv,
    min_freq = 1
)


In [15]:
features_vocab_total_words = len(feature_vocab)
target_vocab_total_words = len(target_vocab)

print(f'Features vocab length: {len(feature_vocab)}')
print(f'Target vocab length: {len(target_vocab)}')

Features vocab length: 2749
Target vocab length: 2747


In [16]:
def make_ngrams(tokenized_text):
    list_ngrams = []
    for i in range(1, len(tokenized_text)):
        list_ngrams.append(tokenized_text[:i+1])
    return list_ngrams


In [17]:
ngrams_list = []
for token in tokenizer_conv:
    ngrams_list.extend(make_ngrams(token))

In [18]:
ngrams_list[:4]

[['what', 'is'],
 ['what', 'is', 'your'],
 ['what', 'is', 'your', 'favorite'],
 ['what', 'is', 'your', 'favorite', 'holiday']]

In [19]:
def add_random_oov_tokens(ngram):
    for idx, word in enumerate(ngram[:-1]):
        if random.uniform(0,1)<0.1:
            ngram[idx] = '<oov>'
        return ngram

In [20]:
ngrams_list_oov = []
for ngrams in ngrams_list:
    ngrams_list_oov.append(add_random_oov_tokens(ngrams))
print(any('<oov>' in ngram for ngram in ngrams_list_oov))

True


In [21]:
def text_to_numerical_seq(tokenized_text):
    tokens_list = []
    if tokenized_text[-1] in target_vocab.get_itos():
        for token in tokenized_text[:-1]:
            num_token = feature_vocab[token] if token in feature_vocab.get_itos() else feature_vocab['<oov>']
            tokens_list.append(num_token)
        num_token = target_vocab[tokenized_text[-1]]
        tokens_list.append(num_token)
        return tokens_list
    return None

In [22]:

input_seq = [text_to_numerical_seq(sequence) for sequence in ngrams_list_oov if text_to_numerical_seq(sequence)]

In [23]:
print(f'Total input sequences: {len(input_seq)}')

Total input sequences: 18562


In [24]:
print(input_seq[7:9])

[[56, 90, 2, 73, 2], [56, 90, 2, 73, 4, 228]]


In [25]:
x = [seq[:-1] for seq in input_seq]
y = [seq[-1] for seq in input_seq]
len(x[0]), y[0]

(1, 11)

In [26]:
print(x[0], y[0])

[12] 11


In [27]:
longest_seq_feature = max(len(seq) for seq in x)
print(longest_seq_feature)

153


In [28]:
padded_x = [F.pad(torch.tensor(seq), (longest_seq_feature - len(seq), 0), value = 0) for seq in x]
padded_x[1], x[2], len(padded_x[1])

(tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  1, 13]),
 [1, 13, 29],
 153)

In [29]:
padded_x = torch.stack(padded_x)
y = torch.tensor(y)


In [30]:
type(y), type(padded_x)

(torch.Tensor, torch.Tensor)

In [31]:
y_one_hot = F.one_hot(y, num_classes = len(target_vocab))

In [32]:
data =  TensorDataset(padded_x, y_one_hot)

In [33]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
batch_size = 32

In [34]:
train_data, test_data = random_split(data, [train_size, test_size])

In [35]:

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle= True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle= False)


In [36]:
class My_BiLSTM(nn.Module):
    def __init__(self, features_vocab_total_words, target_vocab_total_words, embedding_dim, hidden_dim):
        super(My_BiLSTM, self).__init__()
        self.embedding = nn.Embedding(features_vocab_total_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, target_vocab_total_words)

    def forward(self, x):
        x = x.to(self.embedding.weight.device)
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        # Since the LSTM is bidirectional, we concatenate the last hidden state of the forward direction
        # and the first hidden state of the backward direction before passing it to the fully connected layer
        # For batch_first=True, the last timestep of the forward direction is lstm_out[:, -1, :hidden_dim]
        # and the first timestep of the backward direction is lstm_out[:, 0, hidden_dim:]
        output = self.fc(torch.cat((lstm_out[:, -1, :hidden_dim], lstm_out[:, 0, hidden_dim:]), dim=1))
        return output



In [37]:
features_vocab_total_words = 2749
target_vocab_total_words = 2747
embedding_dim = 128
hidden_dim = 200
model = My_BiLSTM(features_vocab_total_words, target_vocab_total_words, embedding_dim = embedding_dim, hidden_dim = hidden_dim)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0009)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
device

device(type='cpu')

In [41]:
model.to(device)

My_BiLSTM(
  (embedding): Embedding(2749, 128)
  (lstm): LSTM(128, 200, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=400, out_features=2747, bias=True)
)

In [42]:
def calculate_topk_accuracy(model, data_loader, k=3):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            output = model(batch_x)

            _, predicted_indices = output.topk(k, dim = 1)

            correct_predictions += torch.any(predicted_indices == torch.argmax(batch_y, dim =1, keepdim = True), dim = 1).sum().item()
            total_predictions += batch_y.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy

In [ ]:
all_accuracies = []
all_loses = []

epochs = 2
for epoch in range(epochs):
    model.train()

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.argmax(dim = 1))
        loss.backward()
        optimizer.step()

    if epochs % 2 == 0:
        accuracy = calculate_topk_accuracy(model, train_loader)
        print(f"Epoch {epoch}/ {epochs}, Loss: {loss.item():.4f}, Train K-accuracy: {accuracy * 100:.2f}%")
        all_accuracies.append(accuracy)
        all_loses.append(loss.item())


In [ ]:
model.parameters()